# Fine-tuning script

*Anusha, Joyce, Nina*

#### Imports

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, DataCollatorForTokenClassification, TrainingArguments
from transformers.models.bert.configuration_bert import BertConfig 
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split

import csv
from Bio import SeqIO
#import evaluate

In [ ]:
# Add ids and comment out others when using
princeton_id = 'aa8417'
#princeton_id = 'ns...'
#princeton_id = 'jf...'

project_dir = f'/scratch/gpfs/{princeton_id}/QCB557_project'

model_name = 'fine_tune_v1'
model_out_dir = f'{project_dir}/models/{model_name}'

#### Load model and tokenizer from Hugging Face

In [ ]:
# use gpu
device = 'cuda:0'

config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
config.num_labels #2 labels
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
model.to(device)

# don't need to move the tokenizer to gpu b/c it's light
# use data collator to pad sequences dynamically during training
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, padding=False)

In [ ]:
print(model)

#### Freeze and unfreeze gradients

We can play around with this later as we imrove the performance of the fine-tuned model.

In [ ]:
# default when using AutoModelForSequenceClassification is that all pretrained parameters/layers are frozen except classification head
# as we go through rounds of fine-tuning, we can optionally unfreeze from of the pretrained layers to improve performance

# unfreeze the last layer in the encoder block
for name, param in model.named_parameters():
    if "encoder.layer.11" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

#### Load data from Hugging Face

In [ ]:
class custom_data_load(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, shuffle=True):
        if shuffle:
            self.dataframe = dataframe.sample(frac=1).reset_index(drop=True)  # shuffle the dataframe
        else:
            self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sequence = self.dataframe.iloc[idx]['sequence']
        label = self.dataframe.iloc[idx]['label']

        # tokenize the sequence
        # tokenizer automatically generates attention masks
        inputs = self.tokenizer(sequence, padding='max_length', max_length=500, truncation=True, return_tensors='pt')
        
        # move inputs to gpu
        inputs = {key: value.to(device) for key, value in inputs.items()}

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

#### Load train and test data

In [ ]:
train_data_to_split = pd.read_csv(f'{project_dir}/data/train.csv')

In [ ]:
test_data = pd.read_csv(f'{project_dir}/data/test.csv')
test_ds = custom_data_load(dataframe = test_data, tokenizer = tokenizer)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer, label_pad_token_id='X')

#### Split training into training and validation

In [ ]:
train_data, valid_data = train_test_split(train_data_to_split, test_size=0.1, random_state=42)

In [ ]:
train_ds = custom_data_load(dataframe = train_data, tokenizer = tokenizer)
valid_ds = custom_data_load(dataframe = valid_data, tokenizer = tokenizer)

#### Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir = model_out_dir,
    num_train_epochs= 10, 
    per_device_train_batch_size=16, # powers of 2
    weight_decay=0.015, # regularization
    learning_rate=1e-5,
    gradient_accumulation_steps=2, # how many batches of gradients are accumulated before parameter update
    gradient_checkpointing=True, # helps reduce memory
    dataloader_num_workers=4,
    
    log_level="error",
    evaluation_strategy="steps",  
    eval_steps=500, 
    save_strategy="epoch",        
    logging_steps=500,
    logging_strategy="steps",
    save_strategy="no",
    fp16=True
    )

#### Extra metrics to compute with validation data

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

#### Train the model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

trainer.train()
trainer.save_model(training_args.output_dir)

In [ ]:
log_df = pd.DataFrame(trainer.state.log_history)
log_df.to_csv(f'{project_dir}/model_logs/log_{model_name}.csv', index=False)